In [1]:
import cv2
import time
import numpy as np
import HandTrackingModule as htm
import os

In [2]:
folderPath = "images/pause"
myList = os.listdir(folderPath)
myList

[]

In [4]:
# Initialize camera
cap = cv2.VideoCapture(0)

# Set camera width and height
w_cam = 1280
h_cam = 720

cap.set(3, w_cam)  # sets width
cap.set(4, h_cam)  # sets height

# Path to save images
folderPath = "images/pause"
os.makedirs(folderPath, exist_ok=True)

pTime = 0

# Initialize hand detector
detector = htm.handDetector(detectionCon=0.75)
pause = False
paused_image_path = os.path.join(folderPath, "current_img.png")

while True:
    success, img = cap.read()
    if not success:
        break

    img = cv2.flip(img, 1)  # Flip the image horizontally

    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)
    tipIds = [4, 8, 12, 16, 20]

    if len(lmList) != 0:
        fingers = []

        # Thumb
        if lmList[tipIds[0]][1] < lmList[tipIds[0] - 1][1]:
            fingers.append(1)
        else:
            fingers.append(0)

        # Other fingers
        for id in range(1, 5):
            if lmList[tipIds[id]][2] < lmList[tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)

        if fingers.count(1) >= 4 and not pause:
            cv2.imwrite(paused_image_path, img)
            pause = True
        elif fingers.count(1) < 4:
            pause = False

    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime

    if pause:
        saved_img = cv2.imread(paused_image_path)
        cv2.putText(saved_img, 'PAUSE', (50, h_cam - 150), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 3)
        cv2.imshow("Image", saved_img)
    else:
        cv2.putText(img, f'FPS={int(fps)}', (50, h_cam - 150), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 3)
        cv2.imshow("Image", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        os.remove(os.path.join(folderPath, "current_img.png"))
        break

cap.release()
cv2.destroyAllWindows()
